In [1]:
import pandas as pd
import numpy as np

In [2]:
# Set state-county association to record states in new DF
county_state = [
    ('maryland','baltimore'),
    ('new jersey','essex'),
    ('illinois','cook'),
    ('new jersey','union'),
    ('virginia','norfolk'),
    ('new york','nassau'),
    ('virginia','harrisonburg'),
    ('virginia','fairfax'),
    ('new jersey','camden'),
    ('texas','harris'),
    ('ohio','franklin'),
    ('indiana','marion'),
    ('georgia','dekalb'),
    ('florida','duval'),
    ('north carolina','wake'),
    ('texas','bexar')
]

In [3]:
# Create new DF to save
county_weather_df = pd.DataFrame()

In [4]:
# Loop through counties and record append data to county_weather_data
for state, county in county_state:
    
    # Load county weather data from csv
    path_to_weather_data = 'data/weather/raw_weather_data_daily/' + county + '.csv'
    weather_df = pd.read_csv(path_to_weather_data,delimiter=',')

    # Add county and state columns with values
    weather_df['county'] = county
    weather_df['state'] = state


    ### Get range of humidity
    # Load hourly humidity data
    humid_df = pd.read_csv("data/weather/raw_weather_data_hourly/" + county + '.csv')[['time','relativehumidity_2m (%)']]
    
    # Remove time signature
    humid_df['time'] = humid_df['time'].str[:-6]

    # Get min, max and avg humidity for each day
    min_humid = humid_df.groupby(['time']).min()['relativehumidity_2m (%)'].values
    max_humid = humid_df.groupby(['time']).max()['relativehumidity_2m (%)'].values
    avg_humid = humid_df.groupby(['time']).mean().round(0)['relativehumidity_2m (%)'].values

    # add humidity to weather_df
    weather_df['min_humidity(%)'] = min_humid
    weather_df['max_humidity(%)'] = max_humid
    weather_df['mean_humidity(%)'] = avg_humid

    # Rename columns
    weather_df = weather_df.rename(columns={
        'time':'date',
        'temperature_2m_mean (°C)':'temp_max(C)',
        'precipitation_sum (mm)':'precip_sum(mm)',
        'windspeed_10m_max (km/h)':'wind_max(km/h)'
        })
    
    # Reorder columns
    weather_df = weather_df[['date','county','state','temp_max(C)','precip_sum(mm)','wind_max(km/h)','min_humidity(%)','max_humidity(%)','mean_humidity(%)']]
    
    county_weather_df = pd.concat([county_weather_df,weather_df])

In [5]:
# Reset index and verify data
county_weather_df = county_weather_df.reset_index().drop(['index'],1)
county_weather_df.head(10)

/var/folders/s9/jd3f2tsn0cgctyvrrylr_kdw0000gn/T/ipykernel_3402/1914905424.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  county_weather_df = county_weather_df.reset_index().drop(['index'],1)


,date,county,state,temp_max(C),precip_sum(mm),wind_max(km/h),min_humidity(%),max_humidity(%),mean_humidity(%)
0,2020-01-01,baltimore,maryland,4.7,0.0,20.9,42,72,54.0
1,2020-01-02,baltimore,maryland,4.2,0.0,18.8,64,84,72.0
2,2020-01-03,baltimore,maryland,9.0,6.8,18.0,68,100,92.0
3,2020-01-04,baltimore,maryland,10.5,10.8,20.6,85,100,95.0
4,2020-01-05,baltimore,maryland,3.6,0.4,33.0,43,92,59.0
5,2020-01-06,baltimore,maryland,4.2,0.0,18.8,46,80,65.0
6,2020-01-07,baltimore,maryland,1.4,8.6,20.8,53,97,76.0
7,2020-01-08,baltimore,maryland,1.3,0.0,29.4,46,90,65.0
8,2020-01-09,baltimore,maryland,-1.0,0.0,18.2,42,67,51.0
9,2020-01-10,baltimore,maryland,5.9,0.0,19.7,71,95,81.0


In [6]:
county_weather_df.dtypes

date                 object
county               object
state                object
temp_max(C)         float64
precip_sum(mm)      float64
wind_max(km/h)      float64
min_humidity(%)       int64
max_humidity(%)       int64
mean_humidity(%)    float64
dtype: object

In [7]:
county_weather_df.to_csv('data/weather/county_weather.csv')